In [8]:
import catboost
import pandas as pd

In [9]:
data = pd.read_csv('pre_data.csv', index_col=0)
df = data[data['DATA_TYPE']=='TRAIN'].drop('DATA_TYPE', axis=1)

In [10]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

/tmp/ipykernel_14968/2001914525.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(2)


,POLICY_BEGIN_MONTH,POLICY_IS_RENEWED,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,POLICY_PRICE_CHANGE
POLICY_BEGIN_MONTH,1.00,-0.04,-0.01,-0.02,0.02,0.07,0.05,-0.02,-0.01,-0.00,0.02,0.00,-0.11,-0.06,0.00
POLICY_IS_RENEWED,-0.04,1.00,0.03,0.01,-0.01,-0.05,-0.04,0.09,0.10,-0.03,-0.05,-0.02,0.11,-0.00,-0.07
POLICY_MIN_AGE,-0.01,0.03,1.00,0.63,-0.06,-0.08,-0.05,-0.09,-0.02,-0.02,-0.03,-0.02,0.03,-0.16,-0.02
POLICY_MIN_DRIVING_EXPERIENCE,-0.02,0.01,0.63,1.00,0.05,-0.07,0.03,-0.07,-0.01,-0.01,-0.03,-0.02,0.05,-0.10,-0.00
VEHICLE_ENGINE_POWER,0.02,-0.01,-0.06,0.05,1.00,-0.07,0.79,0.04,-0.07,0.01,0.00,0.01,-0.07,-0.04,0.04
VEHICLE_IN_CREDIT,0.07,-0.05,-0.08,-0.07,-0.07,1.00,-0.00,-0.13,-0.08,0.01,0.05,0.00,-0.26,-0.11,-0.04
VEHICLE_SUM_INSURED,0.05,-0.04,-0.05,0.03,0.79,-0.00,1.00,-0.01,-0.11,0.02,0.01,0.01,-0.31,-0.06,0.02
CLIENT_HAS_DAGO,-0.02,0.09,-0.09,-0.07,0.04,-0.13,-0.01,1.00,0.22,0.00,-0.00,0.01,0.17,0.01,-0.01
CLIENT_HAS_OSAGO,-0.01,0.10,-0.02,-0.01,-0.07,-0.08,-0.11,0.22,1.00,-0.00,-0.01,0.01,0.15,0.00,0.02
POLICY_COURT_SIGN,-0.00,-0.03,-0.02,-0.01,0.01,0.01,0.02,0.00,-0.00,1.00,0.04,0.04,-0.02,0.00,0.00


In [11]:
X = df.drop('POLICY_IS_RENEWED', axis=1)
y = df['POLICY_IS_RENEWED']

X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77311 entries, 0 to 96604
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   POLICY_BEGIN_MONTH             77311 non-null  int64  
 1   POLICY_SALES_CHANNEL           77311 non-null  object 
 2   POLICY_SALES_CHANNEL_GROUP     77311 non-null  object 
 3   POLICY_BRANCH                  77311 non-null  object 
 4   POLICY_MIN_AGE                 77311 non-null  int64  
 5   POLICY_MIN_DRIVING_EXPERIENCE  77311 non-null  int64  
 6   VEHICLE_MAKE                   77311 non-null  object 
 7   VEHICLE_MODEL                  77311 non-null  object 
 8   VEHICLE_ENGINE_POWER           77311 non-null  float64
 9   VEHICLE_IN_CREDIT              77311 non-null  int64  
 10  VEHICLE_SUM_INSURED            77311 non-null  float64
 11  INSURER_GENDER                 77311 non-null  object 
 12  POLICY_CLM_N                   77311 non-null 

In [12]:
categorical = [1,2,3,6,7,9,11,12,13,14,15,16,17,18,20,23]
numerical = [x for x in range(df.shape[1]-1) if x not in categorical]
y.value_counts()

1    48320
0    28991
Name: POLICY_IS_RENEWED, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
model = catboost.CatBoostClassifier(
    iterations=200,
    loss_function='Logloss',
    eval_metric='AUC',
    use_best_model = True
)

model.fit(
    X_train, y_train,
    cat_features=categorical,
    eval_set=(X_test, y_test),
    plot=True,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.176774
0:	test: 0.6850389	best: 0.6850389 (0)	total: 78.5ms	remaining: 15.6s
1:	test: 0.7010904	best: 0.7010904 (1)	total: 172ms	remaining: 17s
2:	test: 0.6998745	best: 0.7010904 (1)	total: 239ms	remaining: 15.7s
3:	test: 0.7043406	best: 0.7043406 (3)	total: 346ms	remaining: 16.9s
4:	test: 0.7085144	best: 0.7085144 (4)	total: 450ms	remaining: 17.5s
5:	test: 0.7117912	best: 0.7117912 (5)	total: 574ms	remaining: 18.6s
6:	test: 0.7136464	best: 0.7136464 (6)	total: 670ms	remaining: 18.5s
7:	test: 0.7147491	best: 0.7147491 (7)	total: 762ms	remaining: 18.3s
8:	test: 0.7176749	best: 0.7176749 (8)	total: 848ms	remaining: 18s
9:	test: 0.7196149	best: 0.7196149 (9)	total: 944ms	remaining: 17.9s
10:	test: 0.7226821	best: 0.7226821 (10)	total: 1.03s	remaining: 17.8s
11:	test: 0.7261957	best: 0.7261957 (11)	total: 1.11s	remaining: 17.4s
12:	test: 0.7269431	best: 0.7269431 (12)	total: 1.19s	remaining: 17.2s
13:	test: 0.7287247	best: 0.7287247 (13)	total: 1.28s	remaining: 17s
1

In [19]:
model.save_model('ctb')